In [1]:
import pandas as pd
import requests
from azure.storage.blob import BlobServiceClient
import io
import pyarrow.parquet as pq
import pyarrow as pa
import json
import os

# Credenciais
account_name = 'uvabr'
account_key = os.environ['ACCOUNT_KEY']
container_name_send = 'gold'

def extrair_dados_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['dados']
    else:
        print(f"Erro ao acessar a API: {response.status_code}")
        return None

def transformar_dados_para_dataframe(dados):
    df = pd.json_normalize(dados)
    return df

def enviar_parquet_azure(account_name, account_key, container_name_send, file_name, dataframe):
    # Conecte-se ao serviço de armazenamento de blob
    connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Acesse o contêiner
    container_client = blob_service_client.get_container_client(container_name_send)

    # Converta DataFrame para tabela Parquet
    table = pa.Table.from_pandas(dataframe)

    # Crie um buffer de memória para escrever os dados Parquet
    parquet_buffer = io.BytesIO()
    pq.write_table(table, parquet_buffer)

    # Envie os dados para o Azure Blob Storage
    blob_client = container_client.get_blob_client(file_name)
    blob_client.upload_blob(parquet_buffer.getvalue(), overwrite=True)

    print(f"Arquivo Parquet '{file_name}' enviado com sucesso para o Azure Blob Storage.")

def processar_dados_deputados(account_name, account_key, container_name_send):
    url = "https://dadosabertos.camara.leg.br/api/v2/deputados"
    dados = extrair_dados_api(url)
    if dados:
        df = transformar_dados_para_dataframe(dados)
        enviar_parquet_azure(account_name, account_key, container_name_send, 'deputadosPartidos.parquet', df)
        print("Processamento dos dados de deputados concluído.")

# Processar dados dos deputados
processar_dados_deputados(account_name, account_key, container_name_send)


Arquivo Parquet 'deputadosPartidos.parquet' enviado com sucesso para o Azure Blob Storage.
Processamento dos dados de deputados concluído.


In [1]:
#ESSE SCRIPT TEM COMO OBJETIVO DE EXTRAIR TODOS OS DADOS DOS DEPUTADOS E DOS PARTIDOS CRIANDO ARQUIVO deputadosCompleto e partidosCompleto na bronze
# 
# 
import pandas as pd
import requests
import pyarrow as pa
import io
import pyarrow.parquet as pq
from concurrent.futures import ThreadPoolExecutor, as_completed
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient


# Substitua pelos valores reais obtidos no Passo 1
account_name = 'uvabr'
account_key = os.environ['ACCOUNT_KEY']
container_name = 'bronze'

# Conecte-se ao serviço de armazenamento de blob
connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Acesse o contêiner
container_client = blob_service_client.get_container_client(container_name)


# Função para extrair dados da API
def extrair_dados_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['dados']
    else:
        print(f"Erro ao acessar a API: {response.status_code}")
        return None

# Função para transformar dados em dataframe
def transformar_dados_para_dataframe(dados):
    df = pd.json_normalize(dados)
    return df

# Função para extrair dados das URIs em paralelo
def extrair_dados_uris(uris):
    dados_completos = []
    contagem = 0

    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(extrair_dados_api, uri): uri for uri in uris}
        for future in as_completed(future_to_url):
            uri = future_to_url[future]
            try:
                dados_api = future.result()
                if dados_api:
                    dados_completos.append(dados_api)
                contagem += 1
                print(f"Processando {contagem}: {uri}")
            except Exception as e:
                print(f"Erro ao processar {uri}: {e}")

    if dados_completos:
        df_completo = pd.json_normalize(dados_completos)
        return df_completo
    else:
        return pd.DataFrame()
    
def enviar_parquet_azure(account_name, account_key, container_name_send, file_name, dataframe):
    # Conecte-se ao serviço de armazenamento de blob
    connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Acesse o contêiner
    container_client = blob_service_client.get_container_client(container_name_send)

    # Converta DataFrame para tabela Parquet
    table = pa.Table.from_pandas(dataframe)

    # Crie um buffer de memória para escrever os dados Parquet
    parquet_buffer = io.BytesIO()
    pq.write_table(table, parquet_buffer)

    # Envie os dados para o Azure Blob Storage
    blob_client = container_client.get_blob_client(file_name)
    blob_client.upload_blob(parquet_buffer.getvalue(), overwrite=True)

    print(f"Arquivo Parquet '{file_name}' enviado com sucesso para o Azure Blob Storage.")
    
# URL para extrair dados iniciais dos deputados
url = "https://dadosabertos.camara.leg.br/arquivos/deputados/json/deputados.json"
dados = extrair_dados_api(url)
df = transformar_dados_para_dataframe(dados)

# Extrair dados das URIs dos deputados e criar o dataframe df_deputados
df_deputados = extrair_dados_uris(df['uri'])
enviar_parquet_azure(account_name, account_key, 'bronze', f'deputadosCompleto.parquet', df_deputados)

# Remover duplicatas do campo 'ultimoStatus.uriPartido' em df_deputados
uris_partidos_unicos = df_deputados['ultimoStatus.uriPartido'].drop_duplicates()

# Extrair dados das URIs dos partidos e criar um novo dataframe
df_partidos = extrair_dados_uris(uris_partidos_unicos)
enviar_parquet_azure(account_name, account_key, 'bronze', f'partidosCompleto.parquet', df_partidos)

enviar_parquet_azure(account_name, account_key, 'gold', f'deputadosCompleto.parquet', df_deputados)
enviar_parquet_azure(account_name, account_key, 'gold', f'partidosCompleto.parquet', df_partidos)